In [1]:
import os

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(original_dir)))))
    os.chdir(original_dir)
exec(open('setup/default.py').read())
#pd.set_option('display.max_rows', None) # 전체 컬럼 모두 출력


/Users/al01989093/anaconda3/envs/first_pytorch/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# 최근 1년 
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/feature_lab_v1/menu_w_soldout_w_first_menu.csv')
menu_w_sold_out['menu_no'] = menu_w_sold_out.groupby(['day', 'meal_time'])['day'].transform('size')
df = menu_w_sold_out[(menu_w_sold_out.meal_time =='dinner') & 
                     (menu_w_sold_out.menu_no>1)&
                     (menu_w_sold_out.day>='2023-06-01')]


In [17]:
first_menu_info = pd.read_csv(os.getcwd()+'/data/preproc/main/feature_lab_v1/unique_first_menu_w_component.csv')
first_menu_info = first_menu_info[['menu','high_cat','med_cat','ingredient']]
first_menu_info.rename(columns={'menu':'first_menu'}, inplace=True) # 딱 집어서 바꾸고 싶을 떄 

In [23]:
dinner_df = pd.merge(
    df[['post_no','day_of_week','day','course_no','first_menu','is_soldout']],
    first_menu_info,
    how='left',
    on=['first_menu']
)

In [27]:
dinner_df_total = dinner_df.copy()
dinner_df_total['med_cat'] = 'total'
dinner_df_tot_total = dinner_df_total.copy()
dinner_df_tot_total['high_cat'] = 'total'

combined_df = pd.concat([dinner_df, dinner_df_total,dinner_df_tot_total], ignore_index=True)

result = combined_df.groupby(['high_cat', 'med_cat'],dropna=False).agg(
    total=('is_soldout', 'size'),  # 행 수 계산
    soldout=('is_soldout', 'sum')  # soldout의 합계 계산
).reset_index()

result['ratio'] = result['soldout'] / result['total']



In [30]:
result.to_csv(os.getcwd()+'/data/preproc/main/feature_lab_v1/eda/ratio_by_category.csv',index=False)

In [52]:
dinner_df[(dinner_df['high_cat']=='밥') & (pd.isna(dinner_df['med_cat']))]

,post_no,day_of_week,day,course_no,first_menu,is_soldout,high_cat,med_cat,ingredient
22,926901255,Thursday,2023-06-22,[1],느타리버섯무밥,False,밥,NaN,"['버섯', '무']"
25,926901263,Tuesday,2023-06-27,[2],옛날카레라이스,True,밥,NaN,['카레']
46,926901307,Thursday,2023-07-20,[1],느타리버섯무밥,True,밥,NaN,"['버섯', '무']"
101,926901405,Wednesday,2023-09-13,[2],옛날카레라이스,False,밥,NaN,['카레']
113,926901432,Monday,2023-09-25,[2],일식카레 라이스,True,밥,NaN,['카레']
171,926901530,Thursday,2023-11-23,[2],옛날카레라이스,True,밥,NaN,['카레']
193,926901565,Wednesday,2023-12-13,[2],일식커리 라이스,True,밥,NaN,['카레']
206,926901590,Wednesday,2023-12-27,[1],버섯콩나물밥,False,밥,NaN,"['버섯', '콩나물']"
223,926901614,Thursday,2024-01-11,[2],중화풍버섯잡채밥,False,밥,NaN,"['버섯', '잡채']"
253,926901663,Wednesday,2024-02-07,[2],옛날카레라이스,True,밥,NaN,['카레']


In [ ]:
(dinner_df['high_cat']=='밥') and 

In [36]:
dinner_df

,post_no,day_of_week,day,course_no,first_menu,is_soldout,high_cat,med_cat,ingredient
0,926901221,Thursday,2023-06-01,[1],닭갈비,True,NaN,NaN,"['갈비', '닭']"
1,926901221,Thursday,2023-06-01,[2],불고기핫도그,False,핫도그,NaN,['불고기']
2,926901226,Monday,2023-06-05,[1],교자만두칼국수,False,국수,칼,"['교자', '만두']"
3,926901226,Monday,2023-06-05,[2],돈육고추장볶음,False,볶음,NaN,"['돼지', '고추장']"
4,926901228,Wednesday,2023-06-07,[1],취나물영양밥,False,밥,영양,['취나물']
...,...,...,...,...,...,...,...,...,...
367,926901858,Tuesday,2024-05-28,[2],가츠동돈부리,False,돈부리,NaN,"['가츠', '동']"
368,926901860,Wednesday,2024-05-29,[1],북어채해장국,False,국,해장,['북어채']
369,926901860,Wednesday,2024-05-29,[2],산채나물비빔밥,True,밥,비빔,['나물']
370,926901862,Thursday,2024-05-30,[1],소고기샤브샤브,False,NaN,NaN,"['샤브', '소']"
